# 골드 레이어 – 집계 지표 및 거리 계산

이 레이어는 Silver 레이어에서 **최종 분석 데이터셋**을 생성합니다.

## 1. `gold_command_metrics_mv`
- **유형:** 머티리얼라이즈드 뷰  
- **소스:** `silver_combined_events`
- **목적:**  
  - **로봇 ID**, **날짜**, **명령어 이름**별로 로봇 명령 데이터를 집계합니다.
  - 다음을 계산합니다:
    - `command_count` → 실행된 명령어 수
    - `avg_duration_ms` → 명령어별 평균 실행 시간

## 2. `gold_robot_distance`
- **유형:** 테이블  
- **소스:** `silver_combined_events`
- **목적:**  
  - 시간에 따른 위치 변화를 이용해 **로봇 이동 거리**를 계산합니다.
  - 각 로봇의 현재 위치와 이전 위치를 비교하기 위해 윈도우 함수를 사용합니다.
  - 유클리드 거리 공식을 이용해 `step_distance`를 계산합니다.
  - 각 로봇의 첫 번째 행은 거리 = 0으로 기본값이 설정됩니다.

In [0]:
import dlt
from pyspark.sql.functions import col, avg, count, sum as spark_sum, lag, sqrt, pow, to_date
from pyspark.sql.window import Window

CATALOG = "haley_b_demo"
SILVER = "silver"
GOLD = "gold"

@dlt.materialized_view(name=f"{CATALOG}.{GOLD}.gold_command_metrics_mv")
def gold_command_metrics_mv():
    df = dlt.read(f"{CATALOG}.{SILVER}.silver_combined_events")
    df = df.filter(col("command_name").isNotNull())
    df = df.withColumn("date", to_date("timestamp"))
    
    return df.groupBy("robot_id", "date", "command_name").agg(
        count("*").alias("command_count"),
        avg("duration_ms").alias("avg_duration_ms")
    )

In [0]:
import dlt
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, sqrt, pow, col

CATALOG = "haley_b_demo"
SILVER = "silver"
GOLD = "gold"

@dlt.table(
    name=f"{CATALOG}.{GOLD}.gold_robot_distance",
    comment="Calculate robot movement distance based on position changes (window function)"
)
def gold_robot_distance():
    df = dlt.read(f"{CATALOG}.{SILVER}.silver_combined_events")

    # Window for ordering robot movements
    w = Window.partitionBy("robot_id").orderBy("timestamp")

    # Calculate previous position
    df = df.withColumn("prev_x", lag("x").over(w))
    df = df.withColumn("prev_y", lag("y").over(w))

    # Calculate step_distance (Euclidean distance)
    df = df.withColumn(
        "step_distance",
        sqrt(
            pow(col("x") - col("prev_x"), 2) +
            pow(col("y") - col("prev_y"), 2)
        )
    )

    # For the first row, previous value does not exist, so distance = 0
    df = df.fillna({"step_distance": 0})

    return df.select(
        "robot_id",
        "timestamp",
        "x", "y",
        "step_distance"
    )